# 5주차: RAG를 이용한 데이터 설명 챗봇 실습
## 주제: SQLite 스키마 + 생성형 AI로 "데이터 설명 도우미" 만들기

이 노트북에서는 다음을 실습합니다.
1. SQLite 데이터베이스 스키마 정보(테이블, 컬럼) 조회
2. 스키마 정보를 **설명용 텍스트 문서**로 변환
3. 간단한 RAG(Retrieval-Augmented Generation) 구조를 이용해
   - "이 컬럼은 무슨 의미인가요?"
   - "이 테이블은 어떤 용도인가요?" 와 같은 질문에 답하는
   데이터 설명 챗봇을 구현합니다.

> ⚠️ 실제 LLM 호출을 위해서는 OpenAI API Key 또는 다른 LLM API Key가 필요합니다.

## 1. 환경 설정
콜랩(Colab)에서 실행한다고 가정합니다.
- `sqlite3`, `pandas`는 기본 포함
- `openai` 라이브러리는 설치가 필요할 수 있습니다.
- API Key는 수업 시간에 안내한 방법으로 입력하세요.

In [1]:
# 라이브러리 설치 (필요 시)
!pip install --quiet openai pandas

import os
import sqlite3
import pandas as pd
from textwrap import dedent


In [8]:
# 👇 이 셀에서 직접 키를 적지 말고, Colab 메뉴의
#   [실행] → [환경변수 관리] 기능을 활용하는 것을 추천합니다.
# 예시) os.environ["OPENAI_API_KEY"] = "sk-..."  # <-- 실제 키로 교체

from google.colab import userdata

# Colab Secret에서 API Key 로딩
raw_key = userdata.get("OPENAI_API_KEY")
api_key = raw_key.strip() if raw_key else None

if api_key is None:
    raise ValueError(
        "❌ ERROR: Colab userdata에 OPENAI_API_KEY가 없습니다.\n"
        "Colab 왼쪽 메뉴 → 🔑 Secret → OPENAI_API_KEY 추가 후 런타임을 재시작하세요."
    )

# OpenAI 클라이언트 생성
client = OpenAI(api_key=api_key)
print("✅ OpenAI API Key Loaded (prefix):", api_key[:8], "***")

✅ OpenAI API Key Loaded (prefix): sk-proj- ***


## 2. 샘플 SQLite 데이터베이스 만들기
수업에서 이미 사용한 DB가 있다면 그 DB를 사용해도 됩니다.
여기서는 **온라인 쇼핑몰** 예시로 간단한 스키마를 생성합니다.

In [3]:
# 메모리 SQLite DB 생성 (파일로 만들고 싶으면 ':memory:' 대신 'shop.db' 사용)
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# 기존 테이블이 있다면 삭제 (다시 실행할 때를 대비)
cursor.executescript(
    '''
    DROP TABLE IF EXISTS customers;
    DROP TABLE IF EXISTS products;
    DROP TABLE IF EXISTS orders;
    DROP TABLE IF EXISTS order_items;
    '''
)

# 테이블 생성
cursor.executescript(
    '''
    CREATE TABLE customers (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        email TEXT UNIQUE,
        created_at TEXT
    );

    CREATE TABLE products (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        category TEXT,
        price REAL NOT NULL
    );

    CREATE TABLE orders (
        id INTEGER PRIMARY KEY,
        customer_id INTEGER NOT NULL,
        order_date TEXT NOT NULL,
        status TEXT,
        FOREIGN KEY(customer_id) REFERENCES customers(id)
    );

    CREATE TABLE order_items (
        id INTEGER PRIMARY KEY,
        order_id INTEGER NOT NULL,
        product_id INTEGER NOT NULL,
        quantity INTEGER NOT NULL,
        unit_price REAL NOT NULL,
        FOREIGN KEY(order_id) REFERENCES orders(id),
        FOREIGN KEY(product_id) REFERENCES products(id)
    );
    '''
)

# 샘플 데이터 삽입
cursor.executescript(
    '''
    INSERT INTO customers (name, email, created_at) VALUES
        ('홍길동', 'hong@example.com', '2024-01-01'),
        ('김영희', 'kim@example.com', '2024-02-10');

    INSERT INTO products (name, category, price) VALUES
        ('노트북', '전자제품', 1500000),
        ('마우스', '전자제품', 25000),
        ('책상', '가구', 120000);

    INSERT INTO orders (customer_id, order_date, status) VALUES
        (1, '2024-03-01', '완료'),
        (2, '2024-03-05', '배송중');

    INSERT INTO order_items (order_id, product_id, quantity, unit_price) VALUES
        (1, 1, 1, 1500000),
        (1, 2, 2, 25000),
        (2, 3, 1, 120000);
    '''
)
conn.commit()
print('샘플 DB 생성 완료')

샘플 DB 생성 완료


## 3. 스키마 메타데이터 조회하기
`PRAGMA table_info`를 이용해서 각 테이블의 컬럼 정보를 가져옵니다.

In [4]:
def get_tables(conn):
    q = "SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';"
    return [row[0] for row in conn.execute(q)]

def get_table_columns(conn, table_name):
    q = f"PRAGMA table_info({table_name});"
    df = pd.read_sql_query(q, conn)
    return df

tables = get_tables(conn)
print('테이블 목록:', tables)

schema_docs = []
for t in tables:
    cols = get_table_columns(conn, t)
    display(cols)
    schema_docs.append({
        'table': t,
        'columns_df': cols
    })

테이블 목록: ['customers', 'products', 'orders', 'order_items']


,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,1
1,1,name,TEXT,1,None,0
2,2,email,TEXT,0,None,0
3,3,created_at,TEXT,0,None,0


,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,1
1,1,name,TEXT,1,None,0
2,2,category,TEXT,0,None,0
3,3,price,REAL,1,None,0


,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,1
1,1,customer_id,INTEGER,1,None,0
2,2,order_date,TEXT,1,None,0
3,3,status,TEXT,0,None,0


,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,1
1,1,order_id,INTEGER,1,None,0
2,2,product_id,INTEGER,1,None,0
3,3,quantity,INTEGER,1,None,0
4,4,unit_price,REAL,1,None,0


## 4. RAG용 설명 문서 만들기
각 테이블과 컬럼에 대해 **설명 텍스트**를 생성합니다.
- 실제 서비스라면, 사람이 정의한 설명을 넣거나
- AI를 이용해 초안을 만들 수도 있습니다.
여기서는 간단히 템플릿 기반 설명을 생성합니다.

In [5]:
def build_schema_text(table_name, cols_df):
    lines = [f"테이블명: {table_name}"]
    lines.append("컬럼 목록:")
    for _, row in cols_df.iterrows():
        col_name = row['name']
        col_type = row['type']
        not_null = bool(row['notnull'])
        pk = bool(row['pk'])
        desc = f"- {col_name} ({col_type})"
        if pk:
            desc += " ← 기본키"
        if not_null and not pk:
            desc += " (NULL 불가)"
        lines.append(desc)
    return "\n".join(lines)

docs = []
for s in schema_docs:
    text = build_schema_text(s['table'], s['columns_df'])
    docs.append({
        'id': s['table'],
        'content': text
    })

for d in docs:
    print('='*40)
    print(d['content'])

테이블명: customers
컬럼 목록:
- id (INTEGER) ← 기본키
- name (TEXT) (NULL 불가)
- email (TEXT)
- created_at (TEXT)
테이블명: products
컬럼 목록:
- id (INTEGER) ← 기본키
- name (TEXT) (NULL 불가)
- category (TEXT)
- price (REAL) (NULL 불가)
테이블명: orders
컬럼 목록:
- id (INTEGER) ← 기본키
- customer_id (INTEGER) (NULL 불가)
- order_date (TEXT) (NULL 불가)
- status (TEXT)
테이블명: order_items
컬럼 목록:
- id (INTEGER) ← 기본키
- order_id (INTEGER) (NULL 불가)
- product_id (INTEGER) (NULL 불가)
- quantity (INTEGER) (NULL 불가)
- unit_price (REAL) (NULL 불가)


## 5. 간단한 검색 함수 구현 (키워드 기반)
RAG의 Retrieval 부분을 **아주 단순한 키워드 매칭**으로 구현해 봅니다.
- 실제로는 Embedding + 벡터 검색을 쓰는 것이 일반적이지만,
- 교육 목적상 키워드 매칭으로 개념을 이해해 봅니다.

In [6]:
import re
from typing import List, Dict

def simple_retrieval(query: str, docs: List[Dict], top_k: int = 2):
    query_tokens = re.findall(r"\w+", query.lower())
    scored = []
    for d in docs:
        text = d['content'].lower()
        score = sum(token in text for token in query_tokens)
        scored.append((score, d))
    scored = sorted(scored, key=lambda x: x[0], reverse=True)
    return [d for s, d in scored[:top_k] if s > 0]

# 테스트
simple_retrieval("주문과 고객 관계", docs)

[]

## 6. LLM을 이용한 답변 생성 함수
검색된 스키마 설명 텍스트를 컨텍스트로 넣고, LLM에게 질문에 대한 답변을 생성하도록 합니다.

### 프롬프트 설계 포인트
- DB 스키마 설명만을 근거로 답변하도록 지시
- 모르면 모른다고 말하게 하기


In [9]:
def answer_with_schema_llm(question: str, docs: List[Dict]):
    retrieved = simple_retrieval(question, docs, top_k=3)
    context_text = "\n\n".join([d['content'] for d in retrieved])

    system_prompt = (
        "당신은 SQLite 데이터베이스의 스키마를 설명해 주는 조교입니다.\n"
        "아래에 주어진 테이블/컬럼 설명만을 근거로 사용자의 질문에 답하세요.\n"
        "만약 주어진 정보만으로는 답할 수 없다면, 솔직하게 모른다고 말하세요."
    )

    user_prompt = (
        f"[스키마 설명]\n{context_text}\n\n"
        f"[질문]\n{question}"
    )

    completion = client.chat.completions.create(
        model="gpt-4.1-mini",  # 수업 환경에 맞게 변경 가능
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.2,
    )

    return completion.choices[0].message.content

q = "order_items 테이블의 역할은 무엇인가요?"
print(answer_with_schema_llm(q, docs))

order_items 테이블은 주문(order)과 관련된 개별 상품 항목들을 저장하는 역할을 합니다. 각 행은 특정 주문(order_id)에 포함된 하나의 상품(product_id)에 대한 수량(quantity)과 단가(unit_price)를 나타냅니다. 즉, 한 주문에 여러 상품이 있을 때, 각각의 상품 정보를 기록하는 상세 항목 테이블입니다.


## 7. 인터랙티브 질의 실습
이제 자유롭게 질문을 입력해 보세요.
- 컬럼 의미
- 테이블 간 관계
- PK/FK의 역할 등

In [10]:
while True:
    q = input("질문을 입력하세요 (종료하려면 q): ")
    if q.lower() in ["q", "quit", "exit"]:
        break
    answer = answer_with_schema_llm(q, docs)
    print("\n[답변]\n", answer, "\n")

질문을 입력하세요 (종료하려면 q): 테이블의 용도를 설명해줘

[답변]
 테이블에 대한 구체적인 정보가 제공되지 않아, 어떤 테이블을 말씀하시는지 알 수 없습니다. 테이블 이름이나 구조에 대한 정보를 알려주시면 용도를 설명해 드리겠습니다. 

질문을 입력하세요 (종료하려면 q): name 컬럼의 의미는

[답변]
 name 컬럼은 각각 customers 테이블에서는 고객의 이름을, products 테이블에서는 상품의 이름을 의미합니다. 

질문을 입력하세요 (종료하려면 q): customers 테이블을 설명해줘

[답변]
 customers 테이블은 고객 정보를 저장하는 테이블입니다. 주요 컬럼은 다음과 같습니다:

- id: 정수형이며 기본키로 사용됩니다.
- name: 텍스트형이며 NULL 값을 허용하지 않습니다.
- email: 텍스트형이며 이메일 주소를 저장합니다.
- created_at: 텍스트형이며 생성 일시를 저장합니다. 



KeyboardInterrupt: Interrupted by user

## 8. 과제 아이디어
1. 스키마 설명 문장을 더 풍부하게 바꿔 보기 (예: 한국어 자연어로 풀어서 작성)
2. 실제 수업에서 사용한 다른 SQLite DB에 이 코드를 적용해 보기
3. 키워드 기반 Retrieval 대신, Embedding 기반 Retrieval로 확장해 보기
4. "이 질문은 스키마 정보만으로는 답할 수 없습니다"를 판단하는 로직 추가해 보기